In [1]:
# 필요한 모듈 불러오기
import pandas as pd
import numpy as np
import csv
import time

# 난수 발생 패턴을 고정하기 위해 난수는 seed(1234)로 설정
np.random.seed(1234)

# 고정되지 않는 난수를 발생하기 위해서 randomize함수를 이용해 현재시간을 기준으로 난수를 발생하는 함수
def randomize() :
    np.random.seed(time.time)

In [ ]:
# 정규분포 난수값의 평균과 표준편차를 설정하여 가중치의 파라미터를 초기화 할때 사용한다
RND_MEAN = 0
RND_STD = 0.0030

# 학습률을 설정(하이퍼파라미터)
LEARNING_RATE = 0.001

In [ ]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

def load_abalone_dataset():
    with open('abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader,None)
        rows = []
        for row in csvreader:
            rows.append(row)
    
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I' : data[n, 0] = 1
        if row[0] == 'M' : data[n, 1] = 1
        if row[0] == 'F' : data[n, 2] = 1
        data[n, 3:] = row[1:]

def init_model():
    global weight, bias, input_cnt, output_cnt    
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt,output_cnt])
    bias = np.zeros([output_cnt])

def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses, accs = [], []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)

        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss ={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
    
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_train_data():
    